In [ ]:
!pip install --upgrade pip
!python -m pip install pandas numpy scikit-learn tensorflow keras


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 778.6 kB/s eta 0:00:02
   ----------------- ---------------------- 0.8/1.8 MB 873.3 kB/s eta 0:00:02
   ----------------------- ---------------- 1.0/1.8 MB 924.7 kB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 918.8 kB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 939.7 kB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 927.2 kB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\Dell\anaconda3\Anacondaa python\python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow


In [3]:
!pip install scikit-learn


In [4]:
!pip install keras


In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [7]:
df = pd.read_csv("Tweets.csv")   # ← change name if needed

df = df[['text', 'sentiment']]      # keep only required columns
df.dropna(inplace=True)

df.head()


,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['sentiment'], test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [10]:
log_clf = LogisticRegression(max_iter=2000)
log_clf.fit(X_train_tfidf, y_train)
log_pred = log_clf.predict(X_test_tfidf)

print("Logistic Regression Accuracy:", accuracy_score(y_test, log_pred))


Logistic Regression Accuracy: 0.6908660844250364


In [11]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train_tfidf, y_train)
nb_pred = nb_clf.predict(X_test_tfidf)

print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))


Naive Bayes Accuracy: 0.634825327510917


In [12]:
svm_clf = LinearSVC()
svm_clf.fit(X_train_tfidf, y_train)
svm_pred = svm_clf.predict(X_test_tfidf)

print("SVM Accuracy:", accuracy_score(y_test, svm_pred))


SVM Accuracy: 0.6804949053857351


In [13]:
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()


In [14]:
ann = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_dense.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

ann.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

ann.summary()


C:\Users\Dell\anaconda3\Anacondaa python\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │       1,280,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,281 (5.01 MB)

 Trainable params: 1,313,281 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
ann_pred = (ann.predict(X_test_dense) > 0.5).astype("int32").flatten()


172/172 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


In [17]:
print("\n===== ACCURACY COMPARISON =====")
print("Logistic Regression:", accuracy_score(y_test, log_pred))
print("Naive Bayes:", accuracy_score(y_test, nb_pred))
print("SVM:", accuracy_score(y_test, svm_pred))
print("ANN:", accuracy_score(y_test, ann_pred))



===== ACCURACY COMPARISON =====
Logistic Regression: 0.6908660844250364
Naive Bayes: 0.634825327510917
SVM: 0.6804949053857351
ANN: 0.0


In [22]:
%pip install --upgrade tensorflow keras


Note: you may need to restart the kernel to use updated packages.


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['sentiment'], test_size=0.2, random_state=42
)

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Inspecting the result
print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of X_test_tfidf:", X_test_tfidf.shape)

# To see actual feature names
print("First 10 features:", vectorizer.get_feature_names_out()[:10])


Shape of X_train_tfidf: (21984, 5000)
Shape of X_test_tfidf: (5496, 5000)
First 10 features: ['00' '000' '01' '04' '05' '06' '07' '08' '09' '10']
